<a href="https://colab.research.google.com/github/KevinWang676/Bark-Voice-Cloning/blob/main/Bark_Voice_Cloning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/KevinWang676/Bark-Voice-Cloning.git

Cloning into 'Bark-Voice-Cloning'...
remote: Enumerating objects: 544, done.
remote: Counting objects: 100% (182/182), done.
remote: Compressing objects: 100% (97/97), done.
remote: Total 544 (delta 108), reused 155 (delta 83), pack-reused 362
Receiving objects: 100% (544/544), 2.68 MiB | 30.14 MiB/s, done.
Resolving deltas: 100% (295/295), done.


In [ ]:
cd Bark-Voice-Cloning

/content/Bark-Voice-Cloning


In [ ]:
pip install -r requirements.txt

Ignoring fairseq: markers 'platform_system == "Windows"' don't match your environment
Ignoring soundfile: markers 'platform_system == "Windows"' don't match your environment
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 34.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.3/20.3 MB 52.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 72.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!python app.py

2023-10-19 04:02:28.903092: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-10-19 04:02:30 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
smallmodels=False
enablemps=False
offloadcpu=False
forcecpu=False
autolaunch=False


Preloading Models

Loading text model from ./models/text_2.pt to cuda
Loading coarse model from ./models/coarse_2.pt to cuda
Loading fine model from ./models/fine_2.pt to cuda
Launching Bark Voice Cloning UI Server
Running on local URL:  http://127.0.0.1:7861
Running on public URL: https://c3f534bea31be88e9a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
Downloaded HuBERT
en_tokenizer.pth not found. Downloading HuBERT custom tokenizer
Downloaded tokenizer
Loading Hubert ./models/hubert/hubert.pt

Generating Text (1/1) -> file (Seed 40749